## CocktailDB API  
Fun little side project that generates 5 random cocktails, their ID number and all ingredients. 

In [27]:
import requests
import json
import pandas as pd

cocktail_id = [] 
names = []
ingredients = [] 

url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

# Generate 5 random cocktails that are valid
while len(cocktail_id) < 1:
    r = requests.get(url)

    if r.status_code == 200:
        site_json = r.json()

        # Check if the response contains valid data
        if 'drinks' in site_json and site_json['drinks']:
            drink = site_json['drinks'][0]
            name = drink.get('strDrink')

            # Append the columns 
            cocktail_id.append(drink['idDrink'])
            names.append(name)

            # Extract ingredients (removing Nones) -- special case
            # Keeping all arrays equal lenght; joining all ingredients into a single comma-separated list 
            ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]
            ingredients.append(', '.join(filter(None, ingredients_raw)))


        else:
            continue

df_cocktail = pd.DataFrame({"Cocktail ID": cocktail_id, "Cocktail Name": names, "Ingredients":ingredients})
df_cocktail

,Cocktail ID,Cocktail Name,Ingredients
0,12710,Apple Berry Smoothie,"Berries, Apple"


## Improved Version:
Added Number of Ingredients, more to come

In [48]:
import requests
import json
import pandas as pd

cocktail_id = [] 
names = []
ingredients = [] 
num_ingredients = []  
alcoholic = []
acidic = []

url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

# Generate 1 random cocktail that is valid
while len(cocktail_id) < 10:
    r = requests.get(url)

    if r.status_code == 200:
        site_json = r.json()

        # Check if the response contains valid data
        if 'drinks' in site_json and site_json['drinks']:
            drink = site_json['drinks'][0]
            name = drink.get('strDrink')

            # Append the columns 
            cocktail_id.append(drink['idDrink'])
            names.append(name)

            # Extract ingredients (removing Nones) -- special case
            # Keeping all arrays equal length; joining all ingredients into a single comma-separated list 
            ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]
            ingredients.append(', '.join(filter(None, ingredients_raw)))

            # Calculate and append the number of ingredients
            num_ingredients.append(len(list(filter(None, ingredients_raw))))

            alcoholic_status = drink.get('strAlcoholic', '').lower() == 'alcoholic'
            alcoholic.append(alcoholic_status)

            acidic_status = any(ingredient in ["Lemon", "Lime", "Orange"] for ingredient in ingredients_raw)
            acidic.append(acidic_status)

        else:
            continue

# Create DataFrame
df_cocktail = pd.DataFrame({"cocktail_id": cocktail_id, "cocktail_name": names, "num_ingredients": num_ingredients, "ingredients":ingredients, "alcoholic":alcoholic, "acidic":acidic})
df_cocktail.set_index("cocktail_id", inplace=True)

df_cocktail


,cocktail_name,num_ingredients,ingredients,alcoholic,acidic
cocktail_id,,,,,
11375,Foxy Lady,3,"Amaretto, Creme de Cacao, Light cream",True,False
12734,Chocolate Drink,3,"Chocolate, Milk, Water",False,False
178320,Banana Cream Pi,3,"Malibu Rum, Banana Liqueur, Pineapple Juice",True,False
14642,Grim Reaper,3,"Kahlua, 151 proof rum, Grenadine",True,False
15184,San Francisco,4,"Vodka, Creme de Banane, Grenadine, Orange juice",True,False
11011,After Supper Cocktail,3,"Triple sec, Apricot brandy, Lemon juice",True,False
14065,Zippy's Revenge,3,"Amaretto, Rum, Kool-Aid",True,False
12444,Vesuvio,5,"Light rum, Sweet Vermouth, Lemon, Powdered sug...",True,True
17212,Espresso Martini,3,"Vodka, Kahlua, Sugar syrup",True,False


In [47]:
df_cocktail.describe() 

,num_ingredients
count,10.000000
mean,5.000000
std,2.494438
min,2.000000
25%,4.000000
50%,4.500000
75%,5.000000
max,11.000000


In [5]:
csv_path = r"C:\Users\andre\Desktop\DATA583\583_ProjectLocal\CT_test.csv"
df_cocktail.to_csv(csv_path, index=False)


Below this cell is my first attempt. I wasn't aware of two things; 
1) that there's a version of this API that is specifically made to find random cocktails.
2) That iid stands for "ingredient ID" 

Safe to say, lessons learned! Still, I'm including this one because it's part of my process and shows how my approach improved. 

In [3]:

from random import randint

ingredient_id = [] 
descriptions = []

url = "https://www.thecocktaildb.com/api/json/v1/1/lookup.php?iid="

# Generate 5 random ids that are valid
while len(ingredient_id) < 5:
    random_id = randint(1, 700) 

    r = requests.get(url + str(random_id))

    if r.status_code == 200:
        site_json = r.json()

        # Check if the response contains valid data
        if 'ingredients' in site_json and site_json['ingredients']:
            desc = site_json['ingredients'][0]['strDescription']

            if desc is not None: 
                ingredient_id.append(random_id)
                descriptions.append(desc)

        else:
            continue

# Create a DataFrame
df_ingredients = pd.DataFrame({"Cocktail ID": ingredient_id, "Description": descriptions})

df_ingredients

,Cocktail ID,Description
0,505,Schnapps (/ʃnɑːps/ or /ʃnæps/) or schnaps is a...
1,546,"Martini is a brand of Italian vermouth, named ..."
2,570,Scotch whisky (Scottish Gaelic: uisge-beatha n...
3,1,Vodka is a distilled beverage composed primari...
4,64,Schnapps (/ʃnɑːps/ or /ʃnæps/) or schnaps is a...
